In [10]:
import tensorflow as tf
import numpy as np
import time
from sklearn.utils import shuffle
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

<h1>Extract MNIST data</h1>

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",reshape=False,one_hot=True)
tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<h1>Prepare training, validation and testing data</h1>

In [3]:
x_train, y_train           = mnist.train.images, mnist.train.labels
x_validation, y_validation = mnist.validation.images, mnist.validation.labels
x_test, y_test             = mnist.test.images, mnist.test.labels
x_train_pad = np.pad(x_train, ((0,0), (2,2), (2,2), (0,0)), mode='constant', constant_values=0)
x_validation_pad = np.pad(x_validation, ((0,0), (2,2), (2,2), (0,0)), mode='constant', constant_values=0)
x_test_pad = np.pad(x_test, ((0,0), (2,2), (2,2), (0,0)), mode='constant', constant_values=0)

def batches(data_x, data_y, batch_size):
    i = 0  
    while i * batch_size < data_x.shape[0]:
        start = i * batch_size
        end = i * batch_size + batch_size
        yield (data_x[start:end], data_y[start:end])
        i += 1


<h1>Define hyperparameter</h1>

In [4]:
batch_size = 64
num_epochs = 50
lr = 0.001
dropout_rate = 0.6

In [5]:
tf.reset_default_graph()

<h1>Placeholder</h1>

In [6]:
x_in = tf.placeholder(tf.float32, shape=(None, 32, 32, 1))
y_true = tf.placeholder(tf.float32, shape=(None, 10))
training = tf.placeholder_with_default(False, shape=(),name='training')

<h1>Define LeNet-5</h1>

In [7]:
c1 = tf.layers.conv2d(x_in, filters=6, kernel_size=5, padding = 'SAME', activation=tf.nn.relu)


s1 = tf.layers.max_pooling2d(c1, pool_size=[2,2], strides=2)


c2 = tf.layers.conv2d(s1, filters=16, kernel_size=5, activation=tf.nn.relu)


s2 = tf.layers.max_pooling2d(c2, pool_size=[2,2], strides=2)


c3 = tf.layers.conv2d(s2, filters=120, kernel_size=5, activation=tf.nn.relu)


c3_flat = tf.layers.flatten(c3)


d4 = tf.layers.dense(c3_flat, units=84, activation=tf.nn.sigmoid)

d4_drop = tf.layers.dropout(d4,dropout_rate,training=training)

out = tf.layers.dense(d4, units=10)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


<h1>Cost and optimization</h1>

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=out))
opt = tf.train.AdamOptimizer(lr)
minimize = opt.minimize(cost)
correct_pred = tf.equal(tf.argmax(tf.nn.softmax(out),1),tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

<h1>Training, validating, testing</h1>
<h2>1. Print out validation accuracy after each training epoch</h2>
<h2>2. Print out training time on each epoch</h2>
<h2>3. Print out testing accuracy</h2>

In [16]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)   
    for i in range(num_epochs):
        ts = time.time()
        training_batches = batches(x_train_pad, y_train, batch_size)
        for x_data, y_labels in training_batches:
            f_dict = {x_in: x_data, y_true: y_labels, training: True}
            sess.run(minimize, feed_dict=f_dict)
        te = time.time()
        val_acc = sess.run(accuracy, feed_dict={x_in:x_validation_pad, y_true:y_validation})
        print("Epoch " + str(i) + ", Validation Accuracy:{:.4f}".format(val_acc)+ ", Time: {:.3f}s\t".format(te-ts))
       
    test_acc = sess.run(accuracy, feed_dict={x_in:x_test_pad, y_true:y_test})
    print('Test_Accuarcy')
    print(test_acc)

Epoch 0, Validation Accuracy:0.9606, Time: 2.368s	
Epoch 1, Validation Accuracy:0.9822, Time: 2.297s	
Epoch 2, Validation Accuracy:0.9864, Time: 2.292s	
Epoch 3, Validation Accuracy:0.9878, Time: 2.301s	
Epoch 4, Validation Accuracy:0.9898, Time: 2.302s	
Epoch 5, Validation Accuracy:0.9890, Time: 2.323s	
Epoch 6, Validation Accuracy:0.9886, Time: 2.286s	
Epoch 7, Validation Accuracy:0.9914, Time: 2.296s	
Epoch 8, Validation Accuracy:0.9900, Time: 2.299s	
Epoch 9, Validation Accuracy:0.9908, Time: 2.299s	
Epoch 10, Validation Accuracy:0.9900, Time: 2.295s	
Epoch 11, Validation Accuracy:0.9916, Time: 2.295s	
Epoch 12, Validation Accuracy:0.9892, Time: 2.311s	
Epoch 13, Validation Accuracy:0.9910, Time: 2.318s	
Epoch 14, Validation Accuracy:0.9898, Time: 2.316s	
Epoch 15, Validation Accuracy:0.9904, Time: 2.350s	
Epoch 16, Validation Accuracy:0.9902, Time: 2.326s	
Epoch 17, Validation Accuracy:0.9920, Time: 2.301s	
Epoch 18, Validation Accuracy:0.9922, Time: 2.315s	
Epoch 19, Validation A